### Loading libraries

In [ ]:
%%time
import pandas as pd


In [ ]:
import  lightgbm as lgb

#### Reading test and train data

In [ ]:
%%time

df_train = pd.read_csv('../input/train.csv')
df_test = pd.read_csv('../input/test.csv')


#### Displaying head of training and testing

In [ ]:
print(df_train.head(3))
print(df_test.head(3))

#### Define column date as datatype date and define new date features

In [ ]:
# Define column date as datatype date and define new date features
def add_new_features(x):
    x['date'] = pd.to_datetime(x['date'])
    x['year'] = x.date.dt.year
    x['month'] = x.date.dt.month
    x['dayofweek'] = x.date.dt.dayofweek
    return x

#### Dropping date column

In [ ]:
df_train = add_new_features(df_train)
df_test= add_new_features(df_test)


In [ ]:
df_train

### Add new features daily aveage sales and monthly average sales

In [ ]:
df_train['daily_avg']=df_train.groupby(['item','store','dayofweek'])['sales'].transform('mean')
df_train['monthly_avg']=df_train.groupby(['item','store','month'])['sales'].transform('mean')

In [ ]:
daily_avg=df_train.groupby(['item','store','dayofweek'])['sales'].mean().reset_index()
monthly_avg=df_train.groupby(['item','store','month'])['sales'].mean().reset_index()


In [ ]:
monthly_avg

### Merging new features

In [ ]:
def merge(x,y,col,col_name):
    x =pd.merge(x, y, how='left', on=None, left_on=col, right_on=col,
            left_index=False, right_index=False, sort=True,
             copy=True, indicator=False,validate=None)
    
    x=x.rename(columns={'sales':col_name})
    return x

df_test=merge(df_test, daily_avg,['item','store','dayofweek'],'daily_avg')
df_test=merge(df_test, monthly_avg,['item','store','month'],'monthly_avg')


#### Displaying columns in train and testing 

In [ ]:
print(df_test.columns)
print(df_train.columns)

#### Dropping columns

In [ ]:

df_test=df_test.drop(['id'],axis=1)
df_train=df_train.drop(['date'],axis=1)
df_test=df_test.drop(['date'],axis=1)

In [ ]:
df_test.columns

#### INPUT size

In [ ]:
df_train.shape

In [ ]:
df_test.shape

In [ ]:
df_train.head(2)

In [ ]:
df_test.head(2)

### Checking for missing  values

In [ ]:
df_train.isnull().sum()

In [ ]:
df_test.isnull().sum()

### Checking datatypes

In [ ]:
df_train.dtypes

In [ ]:
df_test.dtypes

### Modelling  

In [ ]:
y=pd.DataFrame()
y=df_train['sales']


In [ ]:
df_test.head(2)

In [ ]:
df_train.head()

In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(df_train.drop('sales',axis=1),df_train.pop('sales'),random_state=123,test_size=0.2)

In [ ]:
def XGBmodel(x_train,x_test,y_train,y_test):
    matrix_train = xgb.DMatrix(x_train,label=y_train)
    matrix_test = xgb.DMatrix(x_test,label=y_test)
    model=xgb.train(params={'objective':'reg:linear','eval_metric':'mae'}
                    ,dtrain=matrix_train,num_boost_round=500, 
                    early_stopping_rounds=20,evals=[(matrix_test,'test')],)
    return model

model=XGBmodel(x_train,x_test,y_train,y_test)

%%time
train_data = lgb.Dataset(x,y)
model =lgb.train(param,train_data,)

%%time
output=model.predict(df_test)
result=pd.DataFrame(output)
result

### Submission

In [ ]:
df_test

In [ ]:
df_test1 = pd.read_csv('../input/test.csv')
submission = pd.DataFrame(df_test1.pop('id'))
y_pred = model.predict(xgb.DMatrix(df_test), ntree_limit = model.best_ntree_limit)

submission['sales']= y_pred

submission.to_csv('submission.csv',index=False)

In [ ]:
submission.tail()

In [ ]:

x_test.describe()
